# Sesión 5. Ejemplo de aprendizaje supervisado: Regresión.

# Caso práctico Mantenimiento Predictivo

### Objetivo:
Desarrollar un modelo que sea capaz de predecir la RUL (Remaining Useful Life), en ciclos, de cada motor según los valores obtenidos mediante sensores.

Un segundo objetivo, podría ser clasificar el motor segun si se encuentra en los "Últimos 15 ciclos" o no. Esto correspondería a un ejercicio de clasificación, por lo que no se desarrollará durante esta clase.

# 1. Importar librerías y datos (FD001)

In [ ]:
# load necessary packages and view available data
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import keras
%matplotlib inline
sns.set()
print(os.listdir("Data/mantenimiento"))

In [ ]:
# load data
train= pd.read_csv('Data/mantenimiento/train_FD001.txt', sep=" ", header=None)
test = pd.read_csv('Data/mantenimiento/test_FD001.txt', sep=" ", header=None)
print("train shape: ", train.shape, "test shape: ", test.shape)

In [ ]:
train.head()

# 2. Limpieza de datos y EDA

In [ ]:
# Buscamos NaN
train.isna().sum()

In [ ]:
# Eliminamos las dos últimas columnas (son todas NaN)
train.drop(train.columns[[26, 27]], axis=1, inplace=True)
test.drop(test.columns[[26, 27]], axis=1, inplace=True)

In [ ]:
# El archivo no contiene nombre de variables. Creamos las etiquetas basándonos en la documentación.
index_columns_names =  ["UnitNumber","Cycle"]
op_settings_columns = ["Op_Setting_"+str(i) for i in range(1,4)]
sensor_columns =["Sensor_"+str(i) for i in range(1,22)]
column_names = index_columns_names + op_settings_columns + sensor_columns
print(column_names)

In [ ]:
# name columns
train.columns = column_names
test.columns = column_names
train.head()

### ¿Dispone nuestro dataset de variable objetivo?
En caso negativo, tendremos que calcularla.

**Pregunta:** ¿Como podemos obtener una columna que indique la Remaining Useful Life (RUL)?

In [ ]:
# esta sección calcula la vida útil restante (RUL) en notación T-menos para los datos de entrenamiento
# encuentra el último ciclo por número de unidad
max_cycle = train.groupby('UnitNumber')['Cycle'].max().reset_index()
max_cycle.columns = ['UnitNumber', 'MaxOfCycle']
max_cycle

In [ ]:
# fusionar el ciclo max de nuevo en el df original
train_merged = train.merge(max_cycle, left_on='UnitNumber', right_on='UnitNumber', how='inner')
train_merged

In [ ]:
# calcular el RUL para cada fila
Target_RUL = train_merged["MaxOfCycle"] - train_merged["Cycle"]
train_merged["RUL"] = Target_RUL

# eliminar las columnas innecesarias
train_RUL = train_merged.drop("MaxOfCycle", axis=1)
train_RUL.head(5)

In [ ]:
train_RUL.shape
train_RUL.describe()

**Preguntas:**
* ¿Cuantas unidades tenemos?
* ¿Cual es el valor máximo de vida útil (RUL) de este dataset?
* Si nos dijeran que el motor número 18 es una anomalía, como lo eliminariamos del dataset?

# 3. Visualizar los datos

In [ ]:
# usamos seaborn para visualizar atributos vs variable objetivo (RUL)
explore = sns.PairGrid(data=train_RUL[train_RUL['UnitNumber']<16] ,
                 x_vars='RUL',
                 y_vars=sensor_columns + op_settings_columns,
                 hue="UnitNumber", height=3, aspect=2.5)
explore = explore.map(plt.scatter, alpha=0.5)
explore = explore.set(xlim=(400,0))
explore = explore.add_legend()

**Pregunta:** ¿Qué podemos observar en los gráficos anteriores?

In [ ]:
train_RUL['Op_Setting_3'].describe()

In [ ]:
# la configuración operativa 3 es estable, visualicemos las configuraciones operativas 1 y 2 frente a algunos de los sensores más activos
g = sns.pairplot(data=train_RUL[train_RUL['UnitNumber']<16],
                 x_vars=["Op_Setting_1","Op_Setting_2"],
                 y_vars=["Sensor_2", "Sensor_3", "Sensor_4", "Sensor_7", "Sensor_8", "Sensor_9", "Sensor_11", "Sensor_12", "Sensor_13", "Sensor_14", "Sensor_15", "Sensor_17", "Sensor_20", "Sensor_21"],
                 hue="UnitNumber", aspect=1)

### GRÁFICOS QUE NO PUEDEN FALTAR EN UN EDA:
* Histograma de los ciclos máximos (para este caso en concreto)
* Matriz de correlación
* Boxplot/Histograma por característica

In [ ]:
# Distribution of maximum time cycles
plt.hist(train_RUL.groupby('UnitNumber')['RUL'].max(),bins=20)
plt.xlabel('Max time cycle')

In [ ]:
# Obtener el boxplot para cada uno de los atributos.

atributos_boxplot = train_RUL.plot(kind='box', subplots=True, layout=(8, 4), figsize=(20, 15), sharex=False,
                                 sharey=False, fontsize=8)

In [ ]:
# Obtener los histogramas (distribución de datos) para cada uno de los atributos.

histograma = train_RUL.hist(xlabelsize=10, ylabelsize=10, bins=24, figsize=(15, 15))

In [ ]:
# Correlation matrix
corr = train_RUL.corr()

plt.figure(figsize=(20,20))  
sns.heatmap(corr, annot=True, cmap='coolwarm', center=0)

### ¿Por qué aparecen columnas/filas blancas?

# 4. Preparación de los datos
Esto incluye, entre otros:
1. Limpieza de datos (en caso de que haya NaN)
2. Feature Engineering (eliminar características que son ruido y crear nuevas, si es necesario)
3. Escalado de los datos

**1. Limpieza de datos**

No hay Nan en los datos de entrada y no se eliminarán valores atípicos en este ejemplo. 

In [ ]:
train_RUL.isna().any().any()

**2. Feature Engineering**. 
- Eliminamos aquellas columnas que no nos den información relevante (son ruido y lo único que pueden hacer es empeorar nuestro modelo). También miraremos qué importancia tiene cada característica.

**Según el apartado de visualización:**

Variables a eliminar: S1, S5, S10, S16, S18 y S19

También los settings, ya que hemos visto que no existe ninguna correlación entre ellos y ninguno de los sensores

Antes, pero, vamos a hacer un **ranking de importancia de variables**

Primero eliminamos las siguientes columnas: 'UnitNumber', 'Cycle'

In [ ]:
print(train_RUL.shape)
to_drop_1 = ['UnitNumber', 'Cycle'] + op_settings_columns  
train_features = train_RUL.drop(to_drop_1, axis = 1)
print(train_features.shape)

# set up features and target variable 
y = train_features['RUL']
X = train_features.drop(['RUL'], axis = 1)

In [ ]:
# Usamos un random forest simple para determinar algunas de las características más importantes
# Se puede utilizar para seleccionar características
from sklearn.ensemble import RandomForestRegressor
single_rf = RandomForestRegressor()
single_rf.fit(X, y)
y_pred = single_rf.predict(X)

In [ ]:
# Gráfico de la importancia de las características
importances = single_rf.feature_importances_
indices = np.argsort(importances)[::-1]
feature_names = X.columns    
f, ax = plt.subplots(figsize=(11, 9))
plt.title("Feature ranking", fontsize = 20)
plt.bar(range(X.shape[1]), importances[indices], color="b", align="center")
plt.xticks(range(X.shape[1]), indices) #feature_names, rotation='vertical')
plt.xlim([-1, X.shape[1]])
plt.ylabel("importance", fontsize = 18)
plt.xlabel("index of the feature", fontsize = 18)
plt.show()

# list feature importance
important_features = pd.Series(data=single_rf.feature_importances_,index=X.columns)
important_features.sort_values(ascending=False,inplace=True)
print(important_features.head(22))

* Eliminar Variables con poca importancia.

In [ ]:
# based on the graphs as well as random forest feature importance, I will exclude sensors without much valuable information
print(train_features.shape)
to_drop_2 = ["Sensor_"+str(i) for i in [17, 6, 10, 5, 16, 18, 19, 1]]
train_final = train_features.drop(to_drop_2, axis = 1)
print(train_final.shape)

**3. Transformación (escalado) de los datos**. 
- Primero, dividimos los datos atributos y etiquetas.
- Después, se escalan los datos (en caso de que se crea necesario).
- Finalmente, dividimos los datos en train, test y validación.

Los datos se escalan utilizando el método ``MinMaxScaler()``, que escala y traduce cada atributo individualmente de forma que esté dentro del rango [0, 1]. Esto debe hacerse cuando las escalas de los atributos son diferentes (por ejemplo, Sensor_11 [46.85, 48.53], Sensor_2 [641.21, 644.53] o Sensor_9 [9021.73, 9244.59]).

Primero, dividimos los datos en **atributos**: X (características) y **etiquetas**: y (objetivo).

In [ ]:
# Features X ; Target y 
X = train_final.drop(['RUL'], axis=1) 
y = train_final['RUL']

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Escalamos las características
scaler = MinMaxScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X))
X_scaled.columns = X.columns
X_scaled.head()

# 5. División de los datos
Los datos se dividen en datos de entrenamiento ``X_train``, ``y_train``, datos de validación ``X_val``, ``y_val`` y datos de prueba ``X_test``, ``y_test``.

Funcion ``train_test_split``. Definiremos el porcentaje de los datos de entrada que se utilizaran para validar el modelo. En este ejemplo queremos usar un 55% de los datos de entreno, un 20% de prueba y un 15% de validación.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

test_size = 0.2 # porcentaje de los datos de entrada que utilizaré para validar el modelo

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=test_size)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=test_size)

# 6. Evaluación del Modelo

Importamos modelos:
* ``Linear regresión (como benchmark para comparar)``
* ``Support Vector Regressor``
* ``Random Forest Regressor``
* ``Gradient Boosting Regressor``

In [ ]:
# Importar modelos
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

Otros modelos:
* `'Ridge'`: Ridge()
* `'Lasso'`: Lasso()
* `'KNeighborsRegressor'`:  KNeighborsRegressor()
* `'ExtraTreeRegressor'`: ExtraTreesRegressor()
* `'XGBRegressor'`: XGBRegressor()
* `'MLPRegressor'`: MLPRegressor()

In [ ]:
# Definir folds, metrica de error (ej: balanced_accuracy) y crear modelos
num_folds = 5
error_metrics = {'neg_root_mean_squared_error', 'r2'}

# Lista de modelos a probar
models = {('LR', LinearRegression()), 
          ('SVR', SVR()),
          ('RF', RandomForestRegressor()),
          ('GBR', GradientBoostingRegressor())
         }

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, GridSearchCV

# Entreno Cross-validation

for scoring in error_metrics:
    
    results = [] # guarda los resultados de las métricas de evaluación
    names = []  # Nombre de cada algoritmo
    msg = []  # imprime el resumen del método de cross-validation
    
    print('Evaluation metric: ', scoring)
    
    for name, model in models:
        print('Model ', name)
        cross_validation = KFold(n_splits=num_folds, shuffle=True)
        cv_results = cross_val_score(model, X_train, y_train, cv=cross_validation, scoring=scoring)
        results.append(cv_results)
        names.append(name)
        resume = (name, cv_results.mean(), cv_results.std())
        msg.append(resume)
    print(msg)

    # Compare results between algorithms
    fig = plt.figure()
    fig.suptitle('Compare metric result for algorithms: %s' %scoring)
    ax = fig.add_subplot(111)
    ax.set_xlabel('Candidate models')
    ax.set_ylabel('%s' %scoring)
    plt.boxplot(results)
    ax.set_xticklabels(names)
    plt.show()

# *7. Ajuste de los hiperparámetros*.

Pasos para realizar el ajuste de los hiperparámetros:
* Especificar el modelo(s) a ajustar
* Especificar una métrica a optimizar
* Definir los rangos de los parámetros de búsqueda: *parámetros*
* Asignar un método de validación: *KFold*
* Buscar los hiperparámetros con los datos de validación: *X_val*

### Gradient Boosting

In [ ]:
modelo = GradientBoostingRegressor()
scoring='r2'
params = {
    # Number of trees in random forest
    'n_estimators': [50, 100, 500],  # default=100
    # Maximum features
    'max_features': ['sqrt', 5, 10], # default='sqrt'
     # Maximum number of levels in tree
    'max_depth': [2, 5, None],  #deafult = None
}


# Search for the best combination of hyperparameters
cross_validation = KFold(n_splits=5, shuffle=False)
my_cv = cross_validation.split(X_val)
gsearch = GridSearchCV(estimator=modelo, param_grid=params, scoring=scoring, cv=my_cv)
gsearch.fit(X_val, y_val)

# Print best Result
print("Best result: %f using the following hyperparameters %s" % (gsearch.best_score_, gsearch.best_params_))
means = gsearch.cv_results_['mean_test_score']
stds = gsearch.cv_results_['std_test_score']
params = gsearch.cv_results_['params']

# 8. Evaluación Final del Modelo

In [ ]:
# evaluate metrics on holdout
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(max_depth=2, max_features=5, n_estimators=100)

gbr.fit(X_train, y_train)

y_pred = gbr.predict(X_val)

print("Gradient Boosting Mean Squared Error: ", mean_squared_error(y_val, y_pred))
print("Gradient Boosting Mean Absolute Error: ", mean_absolute_error(y_val, y_pred))
print("Gradient Boosting r-squared: ", r2_score(y_val, y_pred))

In [ ]:
# plot actual vs predicted Remaining Useful Life for the best model
fig, ax = plt.subplots()
ax.scatter(y_val, y_pred, edgecolors=(0, 0, 0))
ax.plot([y_val.min(), y_val.max()], [y_val.min(), y_val.max()], 'k--', lw=4)
ax.set_xlabel('Actual RUL')
ax.set_ylabel('Predicted RUL')
ax.set_title('Remaining Useful Life Actual vs. Predicted')
plt.show()

**¿Cómo podríamos mejorar los resultados?**

Opción: Trabajar solo con datos que tengan una Actual RUL < 150

## Vamos a comprobar con el dataset de Test
También tenemos que importar el archivo de RUL.

In [ ]:
# load data
y_valid = pd.read_csv('Data/mantenimiento/RUL_FD001.txt', sep=" ", header=None)
y_valid

In [ ]:
y_valid.drop(1, axis=1, inplace=True)
y_valid

In [ ]:
X_valid = test.groupby('UnitNumber').last().reset_index()
X_valid

In [ ]:
X_valid = X_valid.drop(to_drop_1 + to_drop_2, axis = 1)

In [ ]:
X_valid_s=scaler.fit_transform(X_valid)

In [ ]:
print(X_valid_s.shape)
print(y_valid.shape)

In [ ]:
X_s=scaler.fit_transform(X)

In [ ]:
# Gradient Boosting Regressor
gbr = GradientBoostingRegressor(max_depth=2, max_features=5, n_estimators=100)

gbr.fit(X_s, y)

y_val_pred = gbr.predict(X_valid_s)

print("Random Forest Mean Squared Error: ", mean_squared_error(y_valid, y_val_pred))
print("Random Forest Mean Absolute Error: ", mean_absolute_error(y_valid, y_val_pred))
print("Random Forest r-squared: ", r2_score(y_valid, y_val_pred))

In [ ]:
# plot actual vs predicted Remaining Useful Life for the best model (GBM)
fig, ax = plt.subplots()
ax.scatter(y_valid, y_val_pred, edgecolors=(0, 0, 0))
ax.plot([y_valid.min(), y_valid.max()], [y_valid.min(), y_valid.max()], 'k--', lw=4)
ax.set_xlabel('Actual RUL')
ax.set_ylabel('Predicted RUL')
ax.set_title('Remaining Useful Life Actual vs. Predicted')
plt.show()